In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.isnull().sum()/len(df)*100

ID      0.0
y       0.0
X0      0.0
X1      0.0
X2      0.0
       ... 
X380    0.0
X382    0.0
X383    0.0
X384    0.0
X385    0.0
Length: 378, dtype: float64

In [ ]:
df.dtypes

ID        int64
y       float64
X0       object
X1       object
X2       object
         ...   
X380      int64
X382      int64
X383      int64
X384      int64
X385      int64
Length: 378, dtype: object

In [ ]:
y = df['y']

In [ ]:
df = df.drop(['y' , 'ID'],axis =1)

In [ ]:
df_num = df.select_dtypes(exclude='object')

In [ ]:
df_num.var() == 0.0

X10     False
X11      True
X12     False
X13     False
X14     False
        ...  
X380    False
X382    False
X383    False
X384    False
X385    False
Length: 368, dtype: bool

In [ ]:
df_num = df_num.drop(df_num.loc[: , df_num.var() == 0.0] , axis = 1)

In [ ]:
df_num.shape

(4209, 356)

In [ ]:
df_num.isnull().sum()/len(df_num)*100

X10     0.0
X12     0.0
X13     0.0
X14     0.0
X15     0.0
       ... 
X380    0.0
X382    0.0
X383    0.0
X384    0.0
X385    0.0
Length: 356, dtype: float64

In [ ]:
df_num.nunique()

X10     2
X12     2
X13     2
X14     2
X15     2
       ..
X380    2
X382    2
X383    2
X384    2
X385    2
Length: 356, dtype: int64

In [ ]:
df_cat = df.select_dtypes(include = 'object')

In [ ]:
df_cat_dum = pd.get_dummies(df_cat)


In [ ]:
df_cat_dum.shape

(4209, 195)

In [ ]:
df_final = pd.concat([df_cat_dum , df_num], axis = 1)

In [ ]:
df_final.head()

,X0_a,X0_aa,X0_ab,X0_ac,X0_ad,X0_af,X0_ai,X0_aj,X0_ak,X0_al,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
x_pca = pca.fit_transform(x)
new_df_pca = pd.DataFrame(x_pca , columns = ['pca1' , 'pca2'])


In [ ]:
print(new_df_pca.explained_var_ratio)

[0.11327864 0.07799109]


In [ ]:
new_df_pca.head()

,pca1,pca2
0,0.850248,-1.252516
1,-0.109302,-1.299658
2,-0.673653,-2.367698
3,-0.480940,-2.695791
4,-0.516369,-2.692792


In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(new_df_pca , y , test_size = 0.8 , random_state=10)

In [ ]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.4, max_depth = 10, alpha = 6, 
                           n_estimators = 20)
model = xgb_reg.fit(x_train , y_train)



In [ ]:
from sklearn.metrics import r2_score , mean_absolute_error
pred_test = model.predict(x_test)
accuracy_test = r2_score(y_test , pred_test)
pred_train = model.predict(x_train)
accuracy_train = r2_score(y_train , pred_train )
mean_abs_error = mean_absolute_error(y_test , pred_test)


In [ ]:
print('accuracy of test' , accuracy_test)
print('accuracy of train' , accuracy_train)
print('mean_absolute_error of test', mean_abs_error)

accuracy of test 0.07124415815741292
accuracy of train 0.8535342261642173
mean_absolute_error of test 9.016447347246835
